In [1]:
import lsdb
import nested_pandas as npd
from astropy.table import QTable, Table
import astropy.units as u
import numpy as np

# Loading Catalogs from Astropy Tables

In [14]:
t = Table()
scale = 100
t['a'] = [1, 4]*scale
t['ra'] = [2.0, 5.0]*scale
t['dec'] = [30.0, 35.0]*scale
t['c'] = ['x', 'y']*scale
t

a,ra,dec,c
int64,float64,float64,str1
1,2.0,30.0,x
4,5.0,35.0,y
1,2.0,30.0,x
4,5.0,35.0,y
1,2.0,30.0,x
4,5.0,35.0,y
1,2.0,30.0,x
4,5.0,35.0,y
1,2.0,30.0,x


In [15]:
lsdb.from_astropy(t)

,a,ra,dec,c
npartitions=1,,,,
"Order: 3, Pixel: 317",int64[pyarrow],double[pyarrow],double[pyarrow],string[pyarrow]


In [16]:
lsdb.from_astropy(t).head(5)

,a,ra,dec,c
_healpix_29,,,,
1430567529421837994,1,2.0,30.0,x
1432121478719349968,4,5.0,35.0,y
1430567529421837994,1,2.0,30.0,x
1432121478719349968,4,5.0,35.0,y
1430567529421837994,1,2.0,30.0,x


In [3]:
# Quantity Table
a = np.array([1, 4, 5], dtype=np.int32)
b = [2.0, 5.0, 8.5]
c = ['x', 'y', 'z']
d = [10, 20, 30] * u.m / u.s
ra = [2.0, 5.0, 10.0]
dec = [-30.0, 35.0, 15.0]

qt = QTable([a, b, c, d, ra, dec],
           names=('a', 'b', 'c', 'd', "RA","DEC"),
           meta={'name': 'first table'})
qt

a,b,c,d,RA,DEC
,,,m / s,,
int32,float64,str1,float64,float64,float64
1,2.0,x,10.0,2.0,-30.0
4,5.0,y,20.0,5.0,35.0
5,8.5,z,30.0,10.0,15.0


In [18]:
lsdb.from_astropy(qt).head(5)

,a,b,c,d,RA,DEC
_healpix_29,,,,,,
1160351551779608234,1,2.0,x,10.0,2.0,-30.0
1432121478719349968,4,5.0,y,20.0,5.0,35.0
1389879706834706546,5,8.5,z,30.0,10.0,15.0


## Multi-Dimensional Columns

### List Columns

In [9]:
# Fixed Size Lists
data = {
        "ra": [10.0, 20.0, 30.0],
        "dec": [-10.0, -20.0, -30.0],
        "spectrum": [[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]],
    }
table = Table(data)
table

ra,dec,spectrum
float64,float64,float64[2]
10.0,-10.0,1.0 .. 2.0
20.0,-20.0,3.0 .. 4.0
30.0,-30.0,5.0 .. 6.0


In [10]:
lsdb.from_astropy(table).head()

,ra,dec,spectrum
_healpix_29,,,
1212933045629049957,10.0,-10.0,[1. 2.]
1176808107119886823,20.0,-20.0,[3. 4.]
2510306432296314470,30.0,-30.0,[5. 6.]


In [11]:
# Ragged Lists
data = {
        "ra": [10.0, 20.0, 30.0],
        "dec": [-10.0, -20.0, -30.0],
        "lc": [[1.0, 2.0], [3.0], [4.0, 5.0, 6.0]],
    }
table = Table(data)
table

ra,dec,lc
float64,float64,object
10.0,-10.0,"[1.0, 2.0]"
20.0,-20.0,[3.0]
30.0,-30.0,"[4.0, 5.0, 6.0]"


In [12]:
lsdb.from_astropy(table).head()

,ra,dec,lc
_healpix_29,,,
1212933045629049957,10.0,-10.0,[1. 2.]
1176808107119886823,20.0,-20.0,[3.]
2510306432296314470,30.0,-30.0,[4. 5. 6.]


### Higher Dimensional Columns

In [15]:
# Higher Dimensional Columns
data = {
        "ra": [10.0, 20.0, 30.0],
        "dec": [-10.0, -20.0, -30.0],
        "images": [
            [[1.0, 2.0], [3.0, 4.0]],
            [[5.0, 6.0], [7.0, 8.0]],
            [[9.0, 10.0], [11.0, 12.0]],
        ],
    }
table = Table(data)
table

ra,dec,images
float64,float64,"float64[2,2]"
10.0,-10.0,1.0 .. 4.0
20.0,-20.0,5.0 .. 8.0
30.0,-30.0,9.0 .. 12.0


In [16]:
# Need to flatten
catalog = lsdb.from_astropy(table, ra_column="ra", dec_column="dec", flatten_tensors=True)
catalog.head()

,ra,dec,images
_healpix_29,,,
1212933045629049957,10.0,-10.0,[1. 2. 3. 4.]
1176808107119886823,20.0,-20.0,[5. 6. 7. 8.]
2510306432296314470,30.0,-30.0,[ 9. 10. 11. 12.]


In [17]:
catalog = lsdb.from_astropy(table, flatten_tensors=False)

NotImplementedError: extension<arrow.fixed_shape_tensor[value_type=double, shape=[2,2]]>